In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
import os
import math
import warnings
import sys
warnings.simplefilter(action='ignore', category=FutureWarning)
excel_file_path = '/content/sample_data/6-7 to 8-9 Sep.xlsx'
xls = pd.ExcelFile(excel_file_path)
available_sheets = xls.sheet_names
Sheets = {
    "Azaad Digital": ["Azaad Anchors", "Azaad Anchors "],
    "Pukhtun Digital": ["Pukhtun Anchors", "Pakhtun Anchors"],
    "Kashmir Digital": ["Kashmir Anchors"],
    "Burak New": ["Burak Anchors", "Burak Anchors "]
}

formattedJournalistData = pd.DataFrame(columns=["Name", "No of Shows", "Views", "Channel", "Date"])
def dateLogicMaker3000(realSheet, designationData, formattedDataFinalized):
    dd = designationData.iloc[::-1].reset_index(drop=True)
    dd.columns = dd.iloc[1]
    dd = dd.drop([0,1]).reset_index(drop=True)
    file_name = os.path.splitext(os.path.basename(excel_file_path))[0]
    allDates = file_name.split("o", 1)[1].strip()
    allDates = " ".join(allDates.split())
    dates = allDates.split(" ")[0].strip()
    month = allDates.split(" ")[1].strip()
    year = allDates.split(" ")[2].strip() if len(allDates.split(" ")) > 2 else "2025"
    dd.columns = dd.columns.str.strip()
    name = dd["Name"]
    date = dates.split("-")
    loopStoper = int(date[2].strip()) if len(date) > 2 else int(date[1].strip())
    selectedDayArray = dataExtractFormat[monthStartDay]
    selectedDayArrayUsed = False
    normalEnteryDate = [3, 2, 2]
    i = 1
    j = 0
    k = 0
    while i <= loopStoper:
        fullDate = f"{i}/{month}/{year}"
        cols = dd.columns.tolist()
        keyColumnSelectors = ["Final Views", "Final Views ", "Final Views 48 Hrs", f"Final Views 48 Hrs({i} & {i+1})", "Total 48 hrs Views", f"48 hrs ({i}&{i+1})", "Views", "48 Hrs", f"48 Hrs ({i} & {i+1})", f"48 hrs ({i}&{i+1})", "48 Hrs Views", "48 Hrs Views "]
        for col in cols:
            found = False
            if col in keyColumnSelectors:
                found = True
                print(f"\nFOUNDED: Column 'Final Views' as {col} in {realSheet} starting from {fullDate}")
                idx = cols.index(col) + 1
                dataPerDate = dd.iloc[:, :idx]
                dd = dd.iloc[:, idx:]
                dataPerDate = dataPerDate.fillna(0)
                if selectedDayArrayUsed == False:
                    if k == len(selectedDayArray):
                        selectedDayArrayUsed = True
                    else:
                        dataBreak = selectedDayArray[k]
                        formattedDataFinalized = enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + selectedDayArray[k]
                        k = k + 1
                if selectedDayArrayUsed == True:
                    if j == 0:
                        dataBreak = normalEnteryDate[j]
                        formattedDataFinalized = enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = j + 1
                    elif j == 1:
                        dataBreak = normalEnteryDate[j]
                        formattedDataFinalized = enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = j + 1
                    elif j == 2:
                        dataBreak = normalEnteryDate[j]
                        formattedDataFinalized = enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = 0
                    else:
                        print("ERROR: In using the normal sequence of date-wise entry")
                        break
                break
        if found == False:
            i = i + 1
    return found, formattedDataFinalized

def enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized):
    properDate = i
    foundShowCol = None
    keyShows = ["No of Shows", "no of Shows"]
    for keyShow in keyShows:
        if keyShow in dataPerDate.columns:
            foundShowCol = keyShow
            break

    if foundShowCol is None:
        print(f"Column 'No of Shows' not found in {realSheet} for {fullDate}, so created it with default value: 1")
        dataPerDate["No of Shows"] = 1
    else:
        dataPerDate.rename(columns={foundShowCol: "No of Shows"}, inplace=True)
    dataPerDate[col] = np.ceil(dataPerDate[col] / dataBreak).astype(int)
    dataPerDate["No of Shows"] = np.ceil(dataPerDate["No of Shows"] / dataBreak).astype(int)
    for l in range(dataBreak):
        newData = pd.DataFrame({
            "Name": name,
            "No of Shows": dataPerDate["No of Shows"],
            "Views": dataPerDate[col],
            "Channel": realSheet,
            "Date": f"{properDate}/{month}/{year}"
        })
        formattedDataFinalized = pd.concat([formattedDataFinalized, newData], ignore_index=True)
        properDate = properDate + 1
    return formattedDataFinalized

print("Enter the number according to the day, month started:")
print("Monday    : 1\nTuesday   : 2\nWednesday : 3\nThursday  : 4\nFriday    : 5\nSaturday  : 6\nSunday    : 7")
monthStartDay = input("Enter number: ")
dataExtractFormat = {
    "1": [2, 2],
    "2": [1, 2],
    "3": [2],
    "4": [1],
    "5": [],
    "6": [2, 2, 2],
    "7": [1, 2, 2]
}
for realSheet, sheetGroup in Sheets.items():
    existingSheet = next((s for s in sheetGroup if s in available_sheets), None)
    if existingSheet:
        df = pd.read_excel(xls, sheet_name=existingSheet)
        print(f"\n\nLOADED SHEET: {existingSheet} as {realSheet}")
        df = df.dropna(how="all").reset_index(drop=True)
        currentHeaders = df.columns.tolist()
        df.columns = df.iloc[0]
        df = pd.concat([pd.DataFrame([currentHeaders], columns=df.columns), df], ignore_index=True)
        rdInvert = df
        rdInvert = rdInvert.iloc[::-1].reset_index(drop=True)
        names = rdInvert["Name"]
        names = names.to_list()
        cut = None
        for row, name in enumerate(names):
            if "Reporters / Vloggers" in name:
                cut = row + 1
                break
        if cut is not None:
            reporterData = rdInvert.iloc[:cut].copy()
            otherData = rdInvert.iloc[cut:].copy()
        else:
            reporterData = None
            otherData = rdInvert
            print(f"NOT FOUND: Reporters data as 'Reporters / Vloggers' in the {realSheet} for reporter")
            break
        names = otherData['Name']
        names = names.to_list()
        cut = None
        for row, name in enumerate(names):
            if "Reporters / Vloggers" in name:
                cut = row + 1
                break
        if cut is not None:
            vloggerData = otherData.iloc[:cut].copy()
            otherData = otherData.iloc[cut:].copy()
        else:
            vloggerData = None
            otherData = otherData
            print(f"NOT FOUND: Reporter data as 'Reporters / Vloggers' in the {realSheet} for vlogger")
        keyAnchors = ["Azaad Defence Anchors ", "Azaad Siasat Anchors ", "Anchors", "Anchors/Analysts"]
        for keyAnchor in keyAnchors:
            names = otherData['Name']
            names = names.to_list()
            cut = None
            for row, name in enumerate(names):
                if keyAnchor == name:
                    cut = row + 1
                    break
            if cut is not None:
                print(f"FOUND: Anchor data as {keyAnchor} in the {realSheet}")
                if keyAnchor == "Anchors/Analysts":
                    anchorData = otherData
                else:
                    anchorData = otherData.iloc[:cut].copy()
                    otherData = otherData.iloc[cut:].copy()
            else:
                anchorData = None
                otherData = otherData
                print(f"NOT FOUND: Anchor data in the {realSheet} as {keyAnchor}")
                continue
            found, formattedJournalistData = dateLogicMaker3000(realSheet, anchorData, formattedJournalistData)
            if found == False:
                print(f"NOT FOUND: Column 'Final Views' in {realSheet} for Anchor")
                break
        if found == False:
            break
    else:
        print(f"NOT FOUND: Channel {realSheet}")
        break


from google.colab import files
print("GOOD JOB: Here is your file, go check your downloads")
formattedJournalistData.to_excel("VloggerDataFormatted.xlsx", index=False)
files.download("VloggerDataFormatted.xlsx")



Enter the number according to the day, month started:
Monday    : 1
Tuesday   : 2
Wednesday : 3
Thursday  : 4
Friday    : 5
Saturday  : 6
Sunday    : 7
Enter number: 1


LOADED SHEET: Azaad Anchors  as Azaad Digital
NOT FOUND: Reporter data as 'Reporters / Vloggers' in the Azaad Digital for vlogger
FOUND: Anchor data as Azaad Defence Anchors  in the Azaad Digital

FOUNDED: Column 'Final Views' as Views in Azaad Digital starting from 1/Sep/2025

FOUNDED: Column 'Final Views' as Views in Azaad Digital starting from 3/Sep/2025

FOUNDED: Column 'Final Views' as 48 Hrs in Azaad Digital starting from 5/Sep/2025

FOUNDED: Column 'Final Views' as 48 Hrs (8 & 9) in Azaad Digital starting from 8/Sep/2025
FOUND: Anchor data as Azaad Siasat Anchors  in the Azaad Digital

FOUNDED: Column 'Final Views' as Views in Azaad Digital starting from 1/Sep/2025

FOUNDED: Column 'Final Views' as Views in Azaad Digital starting from 3/Sep/2025

FOUNDED: Column 'Final Views' as 48 Hrs in Azaad Digital starting 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>